In [ ]:
library(ropls)
library(ggsci)
library(Cairo)
library(tidyverse)
library(malbacR)
library(pmartR)
library(ggplot2)
library(dplyr)
library(tibble)
library(tidyr)
library(paletteer)
library(extrafont)

In [ ]:
group_standard <- 'SFA_group2'
save_file_prex <- 'wilcox_change_SFA_group2'
group_cohort <- 'Validation Cohort'

In [ ]:
path_save <- paste('/mnt/data3/data_exp_0826/result/Validation_Cohort_',save_file_prex,sep = '')
path_save
dir.create(path_save,showWarnings = FALSE)

In [ ]:
data_raw <- readRDS('/mnt/data3/data_exp_0826/result/amide_qcrfsc.rds')

In [ ]:
options(repr.matrix.max.rows = 20,repr.matrix.max.cols = 20)
data_raw$e_data

In [ ]:
data_meta_raw <- read.csv('/mnt/data3/fengyuan/data_exp_0826/meta_all_raw_new.csv') %>% 
    mutate(across(matches("group1|group2"), ~ case_when(
    . == 0 ~ "non-sarcopenia",
    . == 1 ~ "sarcopenia",
    TRUE ~ as.character(.)
  )))
data_meta <- data_meta_raw %>% 
    select(c('ID','group_lv1','time_1','time_2','SMI_group1','SMI_group2','SFA_group1',
             'SFA_group2','VFA_group1','VFA_group2','SMD_group1','SMD_group2')) %>% 
    mutate(across(everything(),.fns = as.character)) %>% 
    pivot_longer(
        cols = c('time_1','time_2'),
        names_to = 'group_lv2',
        values_to = 'Sample') %>% 
    mutate(
        Sample = paste('X',Sample,sep = ''),
        group_lv1 = ifelse(group_lv1 == '1',yes = 'Development Cohort',no = 'Validation Cohort')
    )

In [ ]:
sample_expdata <- data_raw$e_data %>% colnames() %>% .[-1]
sample_expdata[!(sample_expdata %in% (data_meta$Sample %>% unique()))]
sample_expdata[!(sample_expdata %in% (data_meta$Sample %>% unique()))] %>% length()

In [ ]:
data_meta <- data_meta %>% 
    mutate(
        Sample = ifelse(
            test = Sample %in% sample_expdata,
            yes = Sample,
            no = paste('X','0',Sample %>% str_remove('X'),sep = '')
        )
    )
dim(data_meta)
data_meta %>% head()

In [ ]:
data_meta$Sample[!(data_meta$Sample %in% data_raw$f_data$Sample)] %>% length()
data_meta$Sample[!(data_meta$Sample %in% data_raw$f_data$Sample)]

In [ ]:
data_meta[!(data_meta$Sample %in% data_raw$f_data$Sample),]

In [ ]:
options(repr.matrix.max.rows = 12)
f_data_all <- data_raw$f_data
f_data_all

In [ ]:
f_data_time1 <- f_data_all %>% 
    filter(group_lv1 == group_cohort,group_lv2 == 'time_1') %>%
    left_join(data_meta,by = 'Sample') %>% 
    filter(get(group_standard) != '#N/A') %>% 
    arrange(ID)
f_data_time1 %>% dim()

In [ ]:
f_data_time2 <- f_data_all %>% 
    filter(group_lv1 == group_cohort,group_lv2 == 'time_2') %>%
    left_join(data_meta,by = 'Sample') %>% 
    filter(get(group_standard) != '#N/A') %>% 
    arrange(ID)
f_data_time2 %>% dim()

In [ ]:
data_e_time1 <- data_raw$e_data %>% 
  column_to_rownames('Molecule') %>% 
  t() %>% as.data.frame() %>% 
  rownames_to_column('Sample') %>% 
  left_join(f_data_time1 %>% select(c('Sample','ID'))) %>% 
  filter(!is.na(ID)) %>% 
  select(-c('Sample')) %>%
  arrange(ID) %>% 
  column_to_rownames('ID')
data_e_time1 %>% dim()
data_e_time1 %>% head()

In [ ]:
data_e_time2 <- data_raw$e_data %>% 
  column_to_rownames('Molecule') %>% 
  t() %>% as.data.frame() %>% 
  rownames_to_column('Sample') %>% 
  left_join(f_data_time2 %>% select(c('Sample','ID'))) %>% 
  filter(!is.na(ID)) %>% 
  select(-c('Sample')) %>%
  arrange(ID) %>% 
  column_to_rownames('ID')
data_e_time2 %>% dim()
data_e_time2 %>% head()

In [ ]:
dataMatrix_use <- data_e_time2 - data_e_time1
dataMatrix_use <- dataMatrix_use %>% 
    t() %>% as.data.frame() %>% 
    rownames_to_column('Molecule')
dataMatrix_use %>% head()

In [ ]:
f_data_use <- f_data_all %>% 
    filter(group_lv1 == group_cohort,group_lv2 == 'time_2') %>%
    left_join(data_meta,by = 'Sample') %>% 
    arrange(ID) %>%  
    select(-Sample) %>% 
    filter(get(group_standard) != '#N/A') %>% 
    column_to_rownames('ID') %>% 
    rownames_to_column('Sample')
f_data_use %>% dim()
f_data_use %>% head()

In [ ]:
data_raw <- as.metabData(
  e_data = dataMatrix_use,
  f_data = f_data_use,
  e_meta = data_raw$e_meta,
  edata_cname = 'Molecule',
  emeta_cname = 'Molecule',
  fdata_cname = 'Sample'
)

In [ ]:
sampleMetadata <- pmart_amide_norm$f_data
sampleMetadata %>% head()

In [ ]:
sampleMetadata_use <- sampleMetadata %>%
  filter(group_lv1 %in% c(group_cohort),group_lv2 == 'time_2')

In [ ]:
sample_select <- sampleMetadata_use %>%
  pull(Sample) %>% unique()
sample_select %>% length()
sample_select

In [ ]:
dataMatrix_use <- pmart_amide_norm$e_data %>% 
  column_to_rownames('Molecule') %>% 
  t() %>% as.data.frame() %>% 
  rownames_to_column('Sample') %>% 
  filter(Sample %in% sample_select) %>% 
  column_to_rownames('Sample')
dataMatrix_use %>% dim()
dataMatrix_use %>% head()

In [ ]:
file_path <- paste(path_save,'pmart_amide_norm.rds',sep = '/')
file_path
saveRDS(pmart_amide_norm,file_path)
file_path <- paste(path_save,'sampleMetadata_use.csv',sep = '/')
write.csv(sampleMetadata_use,file_path)
file_path

In [ ]:
variableMetadata_use <- pmart_amide_norm$e_meta
variableMetadata_use

In [ ]:
# head(dataMatrix)
options(repr.matrix.max.cols = 20,repr.matrix.max.rows = 10)
dataMatrix_use

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
pca = opls(dataMatrix_use)

### PLSDA analysis

In [ ]:
dataMatrix_use %>% max()
dataMatrix_use %>% min()
group_use

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
results_plsda <- list()
for (i in 1:10) {
  results_plsda[[i]] <- opls(dataMatrix_use, group_use, predI = i, orthoI = 0)
  print(paste("orthoI =", i))
  print(results_plsda[[i]]@summaryDF)
}

best_model <- results_plsda[[which.max(sapply(results_plsda, function(x) x@summaryDF$`Q2(cum)`))]]


In [ ]:
lapply(1:10,function(i){results_plsda[[i]]@summaryDF})

In [ ]:
plsda = opls(dataMatrix_use,group_use,predI = 10,orthoI = 0)


In [ ]:
sample.score = plsda@scoreMN %>% 
  as.data.frame() %>%
  mutate(group_use = group_use)
sample.score %>% head()

In [ ]:
library(vegan)
set.seed(12345678)
result <- adonis2(dataMatrix_use ~ group_use, permutations = 1000, method = "euclidean")
print(result)

In [ ]:
result$`Pr(>F)`[1] %>% round(digits = 3)

In [ ]:
set.seed(12345)
pca_res <- prcomp(dataMatrix_use, scale. = TRUE)
result <- adonis2(pca_res$x ~ group_use, permutations = 999, method = "euclidean")
result

In [ ]:
explained_variance <- plsda@modelDF
pc1_var <- explained_variance[1, "R2X"] * 100  
pc2_var <- explained_variance[2, "R2X"] * 100
options(repr.plot.height = 10,repr.plot.width = 12)
ggplot(sample.score, aes(p1, p2, color = group_use)) +
  geom_hline(yintercept = 0, linetype = 'dashed', linewidth = 0.5) +
  geom_vline(xintercept = 0, linetype = 'dashed', linewidth = 0.5) +
  geom_point(size = 8) +
  annotate(geom = 'text', x = -min(sample.score$p1)*(-0.22), y = 55, label = paste('Permutation test: P = ', result$`Pr(>F)`[1] %>% round(digits = 4),sep = ''), size = 12,hjust = 0.5, vjust = 0) +
  annotate("point", x = -10, y = -10, color = "white") +
  stat_ellipse(level = 0.95, linetype = 'solid', 
               linewidth = 0.5, show.legend = FALSE) +
  scale_color_manual(values = c('#A94322','#2B5C8A')) +
  labs(x = paste0('P1 (', round(pc1_var, 1), '%)'), 
       y = paste0('P2 (', round(pc2_var, 1), '%)')) +
  guides(color = guide_legend(override.aes = list(size = 7))) +
  theme_bw() +
  theme(
    legend.position = c(0.22,0.88),
    legend.text = element_text(color = 'black',size = 40, face = 'plain'),
    legend.title = element_blank(),
    legend.background = element_rect(fill = NA),
    panel.background = element_blank(),
    panel.grid = element_blank(),
    panel.border = element_rect(fill = NA,linewidth = 0.5),
    axis.text = element_text(color = 'black',size = 35, face = 'plain'),
    axis.title = element_text(color = 'black',size = 35, face = 'plain'),
    axis.ticks = element_line(color = 'black'),
    axis.line = element_blank()
  )

In [ ]:
# VIP scores plot
vip.score_plsda = as.data.frame(plsda@vipVn)
colnames(vip.score_plsda) = 'vip'
vip.score_plsda$metabolites = rownames(vip.score_plsda)
vip.score_plsda = vip.score_plsda[order(-vip.score_plsda$vip),]
vip.score_plsda$metabolites = factor(vip.score_plsda$metabolites,
                               levels = vip.score_plsda$metabolites)
 
loading.score_plsda = plsda@loadingMN %>% as.data.frame()
loading.score_plsda$metabolites = rownames(loading.score_plsda)
 
all.score_plsda = merge(vip.score_plsda, loading.score_plsda, by = 'metabolites') %>% 
    arrange(desc(vip))
 
all.score_plsda$cat = paste('A',1:nrow(all.score_plsda), sep = '')
all.score_plsda %>% head()

In [ ]:
library(stringi)
data_plot <- all.score_plsda %>% 
    filter(vip >= 1) %>% 
    arrange(desc(vip)) %>% 
    dplyr::slice(1:25) %>% 
    mutate(metabolites = iconv(metabolites,from = 'GBK',to = 'UTF-8')) %>%
    mutate(
        metabolites = metabolites %>% str_wrap(width = 50,whitespace_only = FALSE)
    ) %>% 
    mutate(metabolites = factor(metabolites,levels = metabolites %>% rev()))
data_plot %>% dim()
data_plot %>% head()

In [ ]:
options(repr.plot.width = 12,repr.plot.height = 15)
ggplot(data_plot, aes(metabolites, vip)) +
  geom_segment(aes(x = metabolites, xend = metabolites,
                   y = 0, yend = vip)) +
  geom_point(shape = 21, size = 5, color = '#008000' ,fill = '#008000') +
  annotate(geom = 'point',x = 1,y = 2.5, color = 'white') +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  scale_y_continuous(expand = c(0,0),limits = c(0,3.5)) +
  labs(x = '', y = 'VIP value') +
  coord_flip() +
  theme_bw() +
  theme(legend.position = 'none',
        legend.text = element_text(color = 'black',size = 12, family = 'Arial', face = 'plain'),
        panel.background = element_blank(),
        panel.grid = element_blank(),
        axis.text = element_text(color = 'black',size = 24, family = 'Arial', face = 'plain'),
        axis.text.x = element_text(angle = 0,hjust = 1),
        axis.title = element_text(color = 'black',size = 20, family = 'Arial', face = 'plain'),
        axis.ticks = element_line(color = 'black'),
        axis.ticks.x = element_blank())

#### oplsda

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
results <- list()
for (i in 0:7) {
  results[[i+1]] <- opls(dataMatrix_use, group_use, predI = 1, orthoI = i)
  print(paste("orthoI =", i))
  print(results[[i+1]]@summaryDF)
}
best_model <- results[[which.max(sapply(results, function(x) x@summaryDF$`Q2(cum)`))]]


In [ ]:
lapply(1:7,function(i){results[[i]]@summaryDF})

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
results <- list()
for (i in 0:7) {
  results[[i+1]] <- opls(dataMatrix_use, group_use, predI = 1, orthoI = i)
  print(paste("orthoI =", i))
  print(results[[i+1]]@summaryDF)
}

best_model <- results[[which.max(sapply(results, function(x) x@summaryDF$`Q2(cum)`))]]


In [ ]:
results[[which.max(sapply(results, function(x) x@summaryDF$`Q2(cum)`))]]

In [ ]:
# OPLS-DA analysis
options(repr.plot.width = 15,repr.plot.height = 15)
dataMatrix_use %>% max()
dataMatrix_use %>% min()
oplsda = opls(dataMatrix_use, group_use, predI = 1, orthoI = 1)#,permI = 200

In [ ]:
# sample scores plot
sample.score = oplsda@scoreMN %>% 
  as.data.frame() %>%
  mutate(group_use = group_use,
         o1 = oplsda@orthoScoreMN[,1])

In [ ]:
sample.score

In [ ]:
library(vegan)
set.seed(12345678)
result <- adonis2(dataMatrix_use ~ group_use, permutations = 1000, method = "euclidean")
print(result)

In [ ]:
result$`Pr(>F)`[1] %>% round(digits = 3)

In [ ]:
explained_variance <- oplsda@modelDF
pc1_var <- explained_variance[1, "R2X"] * 100 
pc2_var <- explained_variance[2, "R2X"] * 100 
options(repr.plot.height = 10,repr.plot.width = 12)
ggplot(sample.score, aes(p1, o1, color = group_use)) +
  geom_hline(yintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_vline(xintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_point(size = 5) +
  annotate(geom = 'text', x = -min(sample.score$p1)*0.14, y = 48, label = paste('Permutation test: P = ', result$`Pr(>F)`[1] %>% round(digits = 4),sep = ''), size = 12,hjust = 0.5, vjust = 0) +
  annotate("point", x = -10, y = -10, color = "white") +
  stat_ellipse(level = 0.95, linetype = 'solid', 
               linewidth = 1, show.legend = FALSE) +
  scale_color_manual(values = c('#A94322','#2B5C8A')) +
  labs(x = paste0('P1 (', round(pc1_var, 1), '%)'), 
       y = paste0('to1 (', round(pc2_var, 1), '%)')) +
  guides(color = guide_legend(override.aes = list(size = 5))) +
  theme_bw() +
  theme(
    legend.position = c(0.22,0.88),
    legend.text = element_text(color = 'black',size = 40, family = 'Arial', face = 'plain'),
    legend.title = element_blank(),
    legend.background = element_rect(fill = NA),
    panel.background = element_blank(),
    panel.grid = element_blank(),
    panel.border = element_rect(fill = NA,linewidth = 1),
    axis.text = element_text(color = 'black',size = 35, family = 'Arial', face = 'plain'),
    axis.title = element_text(color = 'black',size = 35, family = 'Arial', face = 'plain'),
    axis.ticks = element_line(color = 'black'),
    axis.line = element_line(linewidth = 0.8)
  )

In [ ]:
# VIP scores plot
vip.score = as.data.frame(oplsda@vipVn)
colnames(vip.score) = 'vip'
vip.score$metabolites = rownames(vip.score)
vip.score = vip.score[order(-vip.score$vip),]
vip.score$metabolites = factor(vip.score$metabolites,
                               levels = vip.score$metabolites)
vip.score %>% head()

In [ ]:
pvaVn <- apply(dataMatrix_use, 2,
               function(feaVn) cor.test(group_use %>% as.factor() %>% as.numeric(), feaVn)[["p.value"]])

vipVn <- getVipVn(opls(dataMatrix_use, group_use,
                       predI = 1, orthoI = 1,
                       fig.pdfC = "none"))
vipVn %>% head()

In [ ]:
# pvaVn <- apply(dataMatrix_use, 2,
#                function(feaVn) cor.test(group_use %>% as.factor() %>% as.numeric(), feaVn)[["p.value"]])
# p.adj = p.adjust(pvaVn, method = 'BH')
e_meta <- pmart_amide_norm$e_meta %>% as.data.frame() %>% 
    dplyr::rename(metabolites = Molecule)
loading.score = oplsda@loadingMN %>% as.data.frame()
loading.score$metabolites = rownames(loading.score)
all.score  <-  merge(vip.score, loading.score, by = 'metabolites') %>% 
    # mutate(pvalue = pvaVn,p.adj = p.adj) %>% 
    left_join(e_meta ,by = 'metabolites')
all.score$cat = paste('A',1:nrow(all.score), sep = '')
all.score %>% dim()
all.score %>% head()

In [ ]:
library(stringi)
data_plot <- all.score %>% 
    filter(vip >= 1) %>% 
    arrange(desc(vip)) %>% 
    dplyr::slice(1:12,14:21) %>% 
    mutate(metabolites = iconv(metabolites,from = 'GBK',to = 'UTF-8')) %>%
    mutate(
        metabolites = metabolites %>% str_wrap(width = 50,whitespace_only = FALSE)
    ) %>% 
    mutate(metabolites = factor(metabolites,levels = metabolites %>% rev()))
data_plot %>% dim()
data_plot$level  %>% table()
data_plot %>% head()

In [ ]:
options(repr.plot.width = 12,repr.plot.height = 15)
ggplot(data_plot, aes(metabolites, vip)) +
  geom_segment(aes(x = metabolites, xend = metabolites,
                   y = 0, yend = vip)) +
  geom_point(shape = 21, size = 5, color = '#008000' ,fill = '#008000') +
  annotate(geom = 'point',x = 1,y = 2.5, color = 'white') +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  scale_y_continuous(expand = c(0,0),limits = c(0,5)) +
  labs(x = '', y = 'VIP value') +
  coord_flip() +
  theme_bw() +
  theme(legend.position = 'none',
        legend.text = element_text(color = 'black',size = 12, family = 'Arial', face = 'plain'),
        panel.background = element_blank(),
        panel.grid = element_blank(),
        axis.text = element_text(color = 'black',size = 24, family = 'Arial', face = 'plain'),
        axis.text.x = element_text(angle = 0,hjust = 1),
        axis.title = element_text(color = 'black',size = 20, family = 'Arial', face = 'plain'),
        axis.ticks = element_line(color = 'black'),
        axis.ticks.x = element_blank())

In [ ]:
# volcano plot
df = dataMatrix_use %>% as.data.frame()
df$group_use = group_use
df = df[order(df$group_use),]
df = df %>% select(-c('group_use'))
df

In [ ]:
sample_select_m <- sampleMetadata_use$Sample[sampleMetadata_use[[group_standard]] == 'sarcopenia']
sample_select_m_rowid <- match(sample_select_m,rownames(df))
sample_select_f <- sampleMetadata_use$Sample[sampleMetadata_use[[group_standard]] == 'non-sarcopenia']
sample_select_f_rowid <- match(sample_select_f,rownames(df))
sample_select_m_rowid;sample_select_f_rowid

In [ ]:
M.mean = apply(df[sample_select_m_rowid,],2,FUN = mean)
F.mean = apply(df[sample_select_f_rowid,],2,FUN = mean)
FC = M.mean - F.mean
log2FC = FC
# pvalue = apply(df, 2, function(x)
#   {t.test(x[sample_select_m_rowid],x[sample_select_f_rowid])$p.value})
pvalue = apply(df, 2, function(x){wilcox.test(x[sample_select_m_rowid], x[sample_select_f_rowid])$p.value})
p.adj = p.adjust(pvalue, method = 'BH')
p.adj.log = -log10(p.adj)
colcano.df = data.frame(FC,log2FC,pvalue,p.adj, p.adj.log)
colcano.df$cat = ifelse(colcano.df$log2FC >= log2(1.2) & colcano.df$pvalue < 0.05,'Up',
                        ifelse(colcano.df$log2FC <= log2(1/1.2) & colcano.df$pvalue < 0.05,'Down','NS'))
colcano.df$cat %>% table()
colcano.df %>% head()

In [ ]:
e_meta <- pmart_amide_norm$e_meta %>% as.data.frame() %>% 
    dplyr::rename(metabolites = Molecule)
loading.score = oplsda@loadingMN %>% as.data.frame()
loading.score$metabolites = rownames(loading.score)
all.score  <-  merge(vip.score, loading.score, by = 'metabolites') %>% 
    # mutate(pvalue = pvaVn,p.adj = p.adj) %>% 
    left_join(e_meta ,by = 'metabolites')
all.score$cat = paste('A',1:nrow(all.score), sep = '')
all.score %>% dim()
all.score %>% head()

In [ ]:
options(repr.matrix.max.cols = 40)
df_res  <-  colcano.df %>% 
    rownames_to_column('metabolites') %>% 
    left_join(all.score %>% dplyr::select(-c('cat')),by = 'metabolites') %>% 
    left_join(all.score_plsda  %>% select(c('metabolites','vip'))  %>% rename(vip_plsda = vip),by = 'metabolites') %>% 
    dplyr::select(c('HMDB','metabolites','Super.Class','Class','Sub.Class','level',
              'log2FC','pvalue','p.adj','vip','vip_plsda','p1','cat','Annotation','Reactome_id','Reactome_Description')) %>% 
    mutate(
        cat_plsda = case_when(
            vip_plsda > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip_plsda > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip_plsda > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(log2FC) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()
file_path <- paste(path_save,'/df_res_baseline_sarcopenia vs non-sarcopenia.csv',sep = '')
cat(file_path)
write.csv(df_res,file_path)
df_res$cat %>% table()
df_res %>% head()

In [ ]:
options(repr.matrix.max.cols = 40)
df_res  <-  colcano.df %>% 
    rownames_to_column('metabolites') %>% 
    left_join(all.score %>% select(-c('cat')),by = 'metabolites') %>% 
    left_join(all.score_plsda  %>% select(c('metabolites','vip'))  %>% rename(vip_plsda = vip),by = 'metabolites') %>% 
    mutate(
        cat_plsda = case_when(
            vip_plsda > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip_plsda > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip_plsda > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(log2FC) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()
file_path <- paste(path_save,'/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = '')
cat(file_path)
write.csv(df_res,file_path)

In [ ]:
metabolites_select <- c(
    "Isovaleric acid","L-Tryptophan","3a,11b,21-Trihydroxy-20-oxo-5b-pregnan-18-al"
)

In [ ]:
df_res %>% head()

In [ ]:
data_plot <- df_res %>% 
    rename(Molecule = metabolites) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Down','NS','Up'))
    )

In [ ]:
data_plot_label <- data_plot %>% filter(cat %in% c('Up','Down')) %>% 
    arrange(desc(log2FC)) %>% 
    filter(Molecule %in% metabolites_select)

In [ ]:
head(data_plot)

In [ ]:
# tmp <- data_plot %>% left_join(pmart_amide_norm$e_meta,by = 'Molecule')
label_text_down <- data_plot %>% filter(cat == 'Down') %>% group_by(level) %>% summarise(Counts = n()) %>% mutate(res = paste(level,Counts,sep = ': ')) %>% pull(res) %>% paste(collapse = ';')
# tmp <- data_plot %>% left_join(pmart_amide_norm$e_meta,by = 'Molecule')
label_text_up <- data_plot %>% filter(cat == 'Up') %>% group_by(level) %>% summarise(Counts = n()) %>% mutate(res = paste(level,Counts,sep = ': ')) %>% pull(res) %>% paste(collapse = ';')
label_text_down;label_text_up

In [ ]:
library(ggrepel)
options(repr.plot.width = 18,repr.plot.height = 15)
ggplot(data_plot, aes(log2FC, -log10(pvalue),color = cat)) +
    geom_point(size = 4,alpha = 0.6) +
    geom_point(data = data_plot_label, mapping = aes(log2FC, -log10(pvalue),color = cat),size = 8) +
    geom_text_repel(
        data = data_plot_label,
        mapping = aes(label = Molecule),
        color = '#2B5C8A',
        box.padding = 0.5,
        size=10, 
        segment.curvature = 0.5,
        segment.size  = 1,
        force = 80,
        fontface = "italic",
        seed = 42,nudge_x = -2,
        arrow = arrow(length = unit(0.01, "npc")),
        max.overlaps = Inf
    ) +
    annotate(
        geom = 'text',x = -4,y = 4.5,
        label = paste('Low expression in the SFA-Decrease group:\n',label_text_down,sep = ''),
        hjust = 0.1,size = 8) +
    annotate(
        geom = 'text',x = 1.2,y = 4.5,
        label = paste('High expression in the SFA-Decrease group:\n',label_text_up,sep = ''),
        hjust = 0.1,size = 8) +
    geom_vline(xintercept = log2(1.2)) +
    geom_vline(xintercept = -log2(1.2)) +
    geom_hline(yintercept = -log10(0.05)) +
    scale_color_manual(values = c('#2B5C8A','#BABDBC','#A94322')) +
    scale_x_continuous(limits = c(-4,4)) +
    # scale_y_continuous(expand = c(0,0.1),limits = c(0,4.7)) +
    labs(y = '-log10(p-value)') +
    theme_bw() +
    theme(
    legend.position = 'none',
    panel.background = element_blank(),
    panel.grid = element_blank(),
    panel.border = element_rect(fill = NA,linewidth = 1),
    axis.text = element_text(color = 'black',size = 35, family = 'Arial', face = 'plain'),
    axis.title = element_text(color = 'black',size = 35, family = 'Arial', face = 'plain'),
    axis.ticks = element_line(color = 'black'),
    axis.line = element_line(linewidth = 0.8)
    )